# Largrid

In [7]:
using BenchmarkTools
using LinearAlgebra
using DataStructures
using LinearAlgebraicRepresentation
const Lar = LinearAlgebraicRepresentation
const Cells = Vector{Vector{Int}}

Vector{Vector{Int64}} (alias for Array{Array{Int64, 1}, 1})

## grid(sequence::Array{Number,1})::Lar.LAR
Genera un modello LAR 1D. La funzione ritorna due elementi V ed EV che reppresentano rispettivamente la Geometria e la Topologia del modello. 

La funzione prende una sequenza di valori in ingresso e la trasforma in un array con il metodo `collect`.
Attraverso un ciclo for, vengono gradualmente popolati i due array `points` e `hulls` (solo quando i valori di `sequence` sono maggiori di 0), che vengono poi convertiti in Lar.

In [8]:
function grid(sequence...)
	sequence = collect(sequence)
	cursor,points,hulls= (0,[[0.]],[])
	for value in sequence
		points = append!(points, [[cursor + abs(value)]])
		if value>=0
			append!(hulls,[[length(points)-1,length(points)]])
		end
	  cursor += abs(value)
	end
	V = convert(Lar.Points, [p[1] for p in points]')
	EV = convert(Lar.Cells,hulls)
	return V,EV
end

grid (generic function with 1 method)

In [9]:
@benchmark grid(1,-1,1,-1,1,-1,1,-1,1,-1)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.302 μs … 433.287 μs  ┊ GC (min … max): 0.00% … 99.02%
 Time  (median):     2.434 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.796 μs ±  10.514 μs  ┊ GC (mean ± σ):  9.89% ±  2.62%

  ▃▇█▇▆▅▄▄▄▃▂▂▁                                               ▂
  ████████████████▇▇▇▇▆▆▆▆▆▄▅▃▃▁▄▁▄▄▁▅▃▁▁▃▄▃▁▄▃▃▁▅▅▄▄▄▁▄▄▅▅▆▅ █
  2.3 μs       Histogram: log(frequency) by time      4.82 μs <

 Memory estimate: 3.78 KiB, allocs estimate: 51.

### Optimization

In [10]:
@inline function grid_opt(sequence::T...) where T
	cursor,points,hulls= (0,[0.],[])

	for value in sequence
		push!(points, cursor + abs(value))
		if value >= 0
			push!(hulls, [length(points)-1, length(points)])
		end
	    cursor += abs(value)
	end
	V = convert(Lar.Points, points')
	return V, hulls
end

grid_opt (generic function with 1 method)

In [11]:
@benchmark grid_opt(1,-1,1,-1,1,-1,1,-1,1,-1)

BenchmarkTools.Trial: 10000 samples with 191 evaluations.
 Range (min … max):  510.906 ns …  16.673 μs  ┊ GC (min … max):  0.00% … 96.21%
 Time  (median):     529.545 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   615.773 ns ± 992.414 ns  ┊ GC (mean ± σ):  11.10% ±  6.63%

  ▁▇█▇▅▄▄▃▃▂▂▂▂▁                                                ▂
  ███████████████████▇▇▇▇▇▇▆▆▆▆▅▆▆▆▆▄▅▄▅▆▅▃▅▄▅▅▅▅▄▆▄▄▄▄▆▄▄▅▂▃▄▄ █
  511 ns        Histogram: log(frequency) by time        898 ns <

 Memory estimate: 1.12 KiB, allocs estimate: 11.

## qn(n::Int)(sequence::Vector{T})::Lar.LAR
Versione alternativa della funzione `grid` con un parametro di ripetizione `n`. La funzione nidificata `qn0` crea un modello Lar a partire dallo stesso input ricevuto da grid ma ripetendolo n volte.

In [12]:
function qn(n::Int)
	function qn0(sequence::Array{T,1})::Lar.LAR  where T <: Real
		sequence = collect(sequence)
		return Lar.grid(repeat(sequence,outer=n)...)
	end
	return qn0
end

qn (generic function with 1 method)

In [13]:
@benchmark qn(3)([1.5,-2,0.5])

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.037 μs … 736.742 μs  ┊ GC (min … max): 0.00% … 98.55%
 Time  (median):     4.208 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.696 μs ±  15.637 μs  ┊ GC (mean ± σ):  7.37% ±  2.20%

  ▅█▇▆▅▄▃▂▁                                                   ▂
  ██████████████▇▇▆▆▅▄▄▄▄▁▄▃▄▄▄▁▄▅▄▄▅▄▃▁▄▄▄▄▅▄▃▄▄▄▃▄▄▃▃▄▃▃▄▄▄ █
  4.04 μs      Histogram: log(frequency) by time      8.87 μs <

 Memory estimate: 3.75 KiB, allocs estimate: 59.

### Optimization

In [70]:
function qn_opt(n::Int)
	function qn_opt0(sequence::Vector{T})::Lar.LAR  where T <: Real
		return grid_opt(repeat(sequence,outer=n)...)
	end
	return qn_opt0
end

qn_opt (generic function with 1 method)

In [71]:
@benchmark qn_opt(3)([1.5,-2,0.5])

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.749 μs … 589.129 μs  ┊ GC (min … max):  0.00% … 99.08%
 Time  (median):     1.828 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.166 μs ±  11.280 μs  ┊ GC (mean ± σ):  10.34% ±  1.98%

  ██▆▅▄▃▁▁                                                    ▁
  ██████████▇▇▇▆▆▄▅▅▄▅▄▄▅▅▄▅▄▄▅▃▃▅▆▅▅▅▆▅▅▅▄▅▄▅▄▃▂▄▅▄▂▄▄▄▄▂▄▅▄ █
  1.75 μs      Histogram: log(frequency) by time      4.85 μs <

 Memory estimate: 1.78 KiB, allocs estimate: 29.

## grid_0(n::Int)::Matrix{Int64}
Genera un *complesso cellulare 0D* (punti) concatenando 0-celle ugualmente distanziate a intervalli unitari.

La funzione `hcat` si occupa di concatenare array per colonna, riceve in input un array di array contenenti un solo elemento rappresentante un punto.

`grid_0` ritorna questo 0-complesso in forma `Matrix{Int64}`.

In [16]:
function grid_0(n::Int)::Array{Int64,2}
    return hcat([[i] for i in range(0, length=n+1)]...)
end

grid_0 (generic function with 1 method)

In [17]:
@benchmark grid_0(5)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.014 μs … 339.105 μs  ┊ GC (min … max): 0.00% … 98.78%
 Time  (median):     1.051 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.150 μs ±   4.770 μs  ┊ GC (mean ± σ):  5.82% ±  1.40%

  ▂█▆                                                          
  ████▇▆▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▁▁▂▁▂▂▂▂▁▂▂▂▂ ▃
  1.01 μs         Histogram: frequency by time        1.83 μs <

 Memory estimate: 864 bytes, allocs estimate: 12.

### Optimization

In [78]:
function grid_0_opt(n::Int)::Matrix{Int64}
    return hcat([i for i in range(0, length=n+1)]...)
end

grid_0_opt (generic function with 1 method)

In [79]:
@benchmark grid_0_opt(5)

BenchmarkTools.Trial: 10000 samples with 501 evaluations.
 Range (min … max):  221.894 ns …   3.539 μs  ┊ GC (min … max): 0.00% … 93.20%
 Time  (median):     226.615 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   238.493 ns ± 145.263 ns  ┊ GC (mean ± σ):  2.95% ±  4.52%

  ▁▆█▅▃▂▂▃▁▃▂                                                   ▁
  ████████████▇▆▇▇▆▆▆▅▄▅▇▆▅▄▅▅▅▃▆▅▅▅▅▆▆▅▅▆▅▅▅▄▅▄▄▄▅▆▃▄▅▅▄▅▂▅▄▄▅ █
  222 ns        Histogram: log(frequency) by time        334 ns <

 Memory estimate: 224 bytes, allocs estimate: 2.

## grid_1(n::Int)::Matrix{Int64}
Funzione gemella di `grid_0` genera un *complesso cellulare 1D* (segmenti) concatenando 1-celle ugualmente distanziate a intervalli unitari.

Facendo uso della funzione `hcat` concatena array di lunghezza 2, ognuno rappresentante un segmento e le sue due estremità.

`grid_1` ritorna questo 1-complesso in forma `Matrix{Int64}`.

In [20]:
function grid_1(n)
    return hcat([[i,i+1] for i in range(0, length=n)]...)
end

grid_1 (generic function with 1 method)

In [91]:
@benchmark grid_1(5)

BenchmarkTools.Trial: 10000 samples with 102 evaluations.
 Range (min … max):  782.029 ns … 56.070 μs  ┊ GC (min … max):  0.00% … 97.89%
 Time  (median):     797.554 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):   915.850 ns ±  2.212 μs  ┊ GC (mean ± σ):  10.19% ±  4.15%

  ▆█▆▅▄▃▄▃▂▂▁▁                                                 ▂
  ███████████████▇▇▇▆▆▆▆▆▆▇▆▆▆▆▁▅▄▆▆▅▆▆▅▆▅▄▅▄▅▅▅▄▃▅▅▅▃▄▄▃▅▅▆▅▅ █
  782 ns        Histogram: log(frequency) by time      1.35 μs <

 Memory estimate: 864 bytes, allocs estimate: 11.

### Optimization

In [94]:
function grid_1_opt(n::Int)::Matrix{Int64}
    return hcat([[i,i+1] for i in range(0, length=n)]...)
end

grid_1_opt (generic function with 2 methods)

In [95]:
@benchmark grid_1_opt(5)

BenchmarkTools.Trial: 10000 samples with 11 evaluations.
 Range (min … max):  990.727 ns … 486.366 μs  ┊ GC (min … max): 0.00% … 99.37%
 Time  (median):       1.030 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.113 μs ±   4.858 μs  ┊ GC (mean ± σ):  4.34% ±  0.99%

  ▁█▅▂                                                           
  ████▇▅▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▂▂▂▂▂▁▁▂▂▂▂▁▂▂ ▃
  991 ns           Histogram: frequency by time         1.88 μs <

 Memory estimate: 864 bytes, allocs estimate: 11.

## larGrid(n::Int)(d::Int)::Array{Int64,2}
Genera un *complesso cellulare 0D* oppure *un complesso cellulare 1D* a seconda del parametro `d` per cui sono accettati valori dal set ``{0,1}``.

`larGrid` fa uso delle funzioni `grid_0` e `grid_1` per generare i due tipi di complessi.

In [24]:
function larGrid(n::Int)
    function larGrid1(d::Int)::Array{Int64,2}
        if d==0
         return grid_0(n)
        elseif d==1
         return grid_1(n)
        end
    end
    return larGrid1
end

larGrid (generic function with 1 method)

In [25]:
@benchmark larGrid(5)(1)

BenchmarkTools.Trial: 10000 samples with 15 evaluations.
 Range (min … max):  972.067 ns … 255.590 μs  ┊ GC (min … max): 0.00% … 99.11%
 Time  (median):       1.006 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.067 μs ±   2.553 μs  ┊ GC (mean ± σ):  2.37% ±  0.99%

  ▇█▇▆▅▅▄▃▂▁                                                    ▂
  ████████████▇█▇▇█▇▇▇▇▆▆▆▅▆▃▅▁▅▄▄▄▅▃▃▃▄▃▄▄▃▅▄▅▄▄▃▃▄▅▅▄▅▃▄▄▄▄▄▄ █
  972 ns        Histogram: log(frequency) by time       1.96 μs <

 Memory estimate: 864 bytes, allocs estimate: 11.

### Optimization

In [26]:
function larGrid_opt(n::Int)
    function larGrid1_opt(d::Int)::Matrix{Int64}
        if d==0
            return grid_0_opt(n)
        elseif d==1
            return grid_1_opt(n)
        end
    end
    return larGrid1_opt
end

larGrid_opt (generic function with 1 method)

In [27]:
@benchmark larGrid_opt(5)(1)

BenchmarkTools.Trial: 10000 samples with 17 evaluations.
 Range (min … max):  969.000 ns … 261.342 μs  ┊ GC (min … max): 0.00% … 98.22%
 Time  (median):       1.012 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.127 μs ±   4.177 μs  ┊ GC (mean ± σ):  6.34% ±  1.71%

  ▇██▇▅▄▂▁                                                      ▂
  ███████████▇██▇▇▆▆▆▆▅▅▅▆▅▆▅▄▅▄▄▂▂▄▃▃▄▄▄▄▃▄▅▃▅▄▂▅▅▆▄▅▄▅▄▃▄▄▅▄▄ █
  969 ns        Histogram: log(frequency) by time       2.14 μs <

 Memory estimate: 864 bytes, allocs estimate: 11.

## cart(args::Array{Array{Any,1},1})::Array{Tuple,1}
Funzione che esegue il prodotto cartesiano delle collezioni inserite nell' array in input. Restituisce un `Array` di `Tuple`. Il numero di 
 unary `Array` argument. Return an `Array` of `Tuple`. The number `Tuple` restituito equivale al prodotto delle dimensione delle collezioni date in input.
 
Il metodo `product` ritorna un iteratore del prodotto di vari iteratori. Restituisce tutte le combinazioni possibili degli elementi delle collezioni in input sottoforma di `Tuple`.
`collect` trasforma l'output di `product` in un array i cui elementi vengono concatenati per riga dal metodo `vcat`.


In [28]:
function cart(args)::Array{Tuple,1}
    return sort(vcat(collect(Iterators.product(args...))...))
end

cart (generic function with 1 method)

In [29]:
@benchmark cart([[1,2],["a"],[3,4]])

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):  7.420 μs …  1.204 ms  ┊ GC (min … max): 0.00% … 97.99%
 Time  (median):     7.662 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.989 μs ± 12.038 μs  ┊ GC (mean ± σ):  1.48% ±  0.98%

  ▄██▆▄▂▁      ▁                                             ▂
  █████████▇▇▇███▇▇▆▅▄▄▁▄▄▃▃▃▃▃▄▁▄▅▄▄▃▁▁▅▄▄▃▅▃▃▄▅▄▃▄▄▁▅▄▅▃▄▄ █
  7.42 μs      Histogram: log(frequency) by time     14.2 μs <

 Memory estimate: 1.81 KiB, allocs estimate: 28.

### Optimization

In [30]:
function cart_opt(args)
    return sort(reduce(vcat, collect(Iterators.product(args...))))
end

cart_opt (generic function with 1 method)

In [31]:
@benchmark cart_opt([[1,2],["a"],[3,4]])

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):  7.213 μs …  1.086 ms  ┊ GC (min … max): 0.00% … 98.23%
 Time  (median):     7.432 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.760 μs ± 10.870 μs  ┊ GC (mean ± σ):  1.37% ±  0.98%

  ▆█▇▅▄▂▁▁                                                   ▂
  █████████▇██▇███▇▆▅▆▅▅▄▃▃▄▃▅▄▄▄▄▄▅▃▄▅▅▁▄▃▃▄▄▁▄▃▄▃▁▄▅▃▅▃▃▃▅ █
  7.21 μs      Histogram: log(frequency) by time       14 μs <

 Memory estimate: 1.72 KiB, allocs estimate: 21.

## larVertProd(vertLists::Array{Points,1})::Points
Genera le coordinate intere (0-celle) di una *griglia multidimensionale*
Viene effettuato il prodotto cartesiano dei vertici contenuti in `vertLists` con la funzione `cart`. Le tuple restituite sono poi trasformate in formato Array per ottenere `coords`, una collezione di array bidimensionali. L'output è poi modificato nella forma dalle funzioni `hcat` e `sortslices`.

Sono messe a disposizione due versione della funzione `larVertProd`, che accettano rispettivamente `Array{Array{Int64,2},1}` e `Array{Float64,2},1}` come argomento

In [32]:
function larVertProd(vertLists::Array{Array{Int64,2},1})::Array{Int64,2}
   coords = [[x[1] for x in v] for v in Lar.cart(vertLists)]
   return sortslices(hcat(coords...), dims=2)
end
function larVertProd(vertLists::Array{Array{Float64,2},1})::Array{Float64,2}
   coords = [[x[1] for x in v] for v in Lar.cart(vertLists)]
   return sortslices(hcat(coords...), dims=2)
end

larVertProd (generic function with 2 methods)

In [33]:
@benchmark larVertProd([larGrid(2)(0), larGrid(2)(0)])

BenchmarkTools.Trial: 10000 samples with 5 evaluations.
 Range (min … max):  6.393 μs …  1.006 ms  ┊ GC (min … max): 0.00% … 98.05%
 Time  (median):     6.687 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.541 μs ± 24.215 μs  ┊ GC (mean ± σ):  7.73% ±  2.40%

  ▆█▇▅▄▂▁ ▁▁                                                 ▁
  ██████████▇▇▇▅▅▃▄▄▃▄▄▅▂▄▄▃▄▄▃▄▂▂▃▂▃▃▄▃▃▃▃▄▄▄▄▃▃▃▃▃▂▄▂▃▂▄▃▄ █
  6.39 μs      Histogram: log(frequency) by time     15.5 μs <

 Memory estimate: 5.28 KiB, allocs estimate: 76.

### Optimization

In [34]:
function larVertProd_opt(vertLists::Vector{Matrix{Int64}})::Matrix{Int64}
    coords = [[x[1] for x in v] for v in Lar.cart(vertLists)]
    return sortslices(hcat(coords...), dims=2)
end
function larVertProd_opt(vertLists::Vector{Matrix{Float64}})::Matrix{Float64}
    coords = [[x[1] for x in v] for v in Lar.cart(vertLists)]
    return sortslices(hcat(coords...), dims=2)
end

larVertProd_opt (generic function with 2 methods)

In [35]:
@benchmark larVertProd_opt([larGrid_opt(2)(0), larGrid_opt(2)(0)])

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.939 μs … 653.107 μs  ┊ GC (min … max): 0.00% … 97.32%
 Time  (median):     5.169 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.799 μs ±  16.565 μs  ┊ GC (mean ± σ):  7.43% ±  2.58%

  ▅█▇▆▅▃▂▁▁▁▁                                                 ▁
  ████████████▇▇▆▅▆▄▅▂▄▂▂▄▄▄▄▄▄▃▄▄▄▂▂▄▂▄▃▃▄▃▃▄▄▄▄▄▃▃▃▃▂▂▄▅▄▄▄ █
  4.94 μs      Histogram: log(frequency) by time      11.2 μs <

 Memory estimate: 4.56 KiB, allocs estimate: 62.

## index2addr(shape::Array{Int64,1})(multiIndex)::Int
Funzione che trasforma un *Multi-index in un indirizzo*. Multi-index è un generalizzazione del concetto di trasformazione di un indice intero in un *insieme ordinato di tuple di indici*. 
La funzione `index2addr` trasforma una lista di `shape` per un *array multidimensionale* in una funzione che, quando applicata ad un *array multi-index*, ad esempio una lista di interi di `Tuple` senza limiti di `shape`, ritorna gli *indirizzi interi* delle componenti dell'array corrispondenti all'interno della *memoria lineare* dell'array multidimensionale.


In [36]:
function index2addr( shape::Array{Int64,1} )
    n = length(shape)
    theShape = append!(shape[2:end],1)
    weights = [prod(theShape[k:end]) for k in range(1, length=n)]

    function index2addr0( multiIndex::Array{Int,1} )::Int
        return dot(collect(multiIndex), weights) + 1
    end

    return index2addr0
end

index2addr (generic function with 1 method)

In [37]:
@benchmark index2addr([2,7])([1,4])

BenchmarkTools.Trial: 10000 samples with 226 evaluations.
 Range (min … max):  330.540 ns …  24.412 μs  ┊ GC (min … max):  0.00% … 96.85%
 Time  (median):     354.604 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   433.524 ns ± 958.446 ns  ┊ GC (mean ± σ):  11.97% ±  5.34%

  ▅██▇▆▅▃▂▁▁       ▁                                            ▂
  ██████████████████████▇██▇██▇▆▆▆▇▇▆▇▆▇▆▆▇▆▆▆▆▆▅▆▆▆▅▆▅▅▅▄▃▄▃▃▅ █
  331 ns        Histogram: log(frequency) by time        853 ns <

 Memory estimate: 608 bytes, allocs estimate: 8.

### Optimization

In [38]:
@inline function index2addr_opt( shape::Vector{Int64} )
    n = length(shape)
    theShape = append!(shape[2:end],1)
    weights = [prod(theShape[k:end]) for k in range(1, length=n)]

    function index2addr0( multiIndex::Vector{Int} )::Int
        return dot(collect(multiIndex), weights) + 1
    end

    return index2addr0
end

index2addr_opt (generic function with 1 method)

In [39]:
@benchmark index2addr_opt([2,7])([1,4])

BenchmarkTools.Trial: 10000 samples with 229 evaluations.
 Range (min … max):  327.454 ns …  20.136 μs  ┊ GC (min … max):  0.00% … 97.15%
 Time  (median):     348.731 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   428.117 ns ± 945.485 ns  ┊ GC (mean ± σ):  12.00% ±  5.34%

  ▄██▇▆▅▄▃▃▂▁▂▂▁▁▁▁▁▁▁  ▁     ▁  ▁                              ▂
  ██████████████████████████▇██▇▇██▇▇██▇▆▆▇▇▇▇▇▆▆▇▆▄▅▆▅▆▆▄▁▄▄▃▆ █
  327 ns        Histogram: log(frequency) by time        791 ns <

 Memory estimate: 608 bytes, allocs estimate: 8.

## larCellProd(cellLists::Array{Cells,1})::Cells
Genera una *griglia cellulare* dal *prodotto Cartesiano* di complessi 0/1-dimensionali.
Il complesso di output è generato dal prodotto di un qualsiasi numero di complessi cellulari 0- oppure 1-dimensionali. Il prodotto di ``d`` complessi 1-dimensionali generano un *solido ``d``-cellulare*, mentre il prodotto di ``n`` complesso 0-dimensionali e ``n-d`` complessi 1-dimensionali (``d < n``) generano un *Non-solido ``(n-d)``-cellulare*, inserito nel spazio ``n``-dimensionale.

In [40]:
function larCellProd(cellLists::Array{Cells,1})::Cells
    shapes = [length(item) for item in cellLists]
    subscripts = cart([collect(range(0, length=shape)) for shape in shapes])
    indices = [collect(tuple) .+ 1 for tuple in subscripts]
 
    jointCells = [cart([cells[k] for (k,cells) in zip(index,cellLists)]) for index in indices]
    convertIt = index2addr([ (length(cellLists[k][1]) > 1) ? shape .+ 1 : shape for (k,shape) in enumerate(shapes) ])
    [vcat(map(convertIt, map(collect,jointCells[j]))...) for j in 1:length(jointCells)]
end

larCellProd (generic function with 1 method)

In [41]:
c1 = [[0,1],[1,2],[2,3]]
c0 = [[0],[1],[2]]
@benchmark larCellProd([c1, c1, c0])

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  73.166 μs …   5.341 ms  ┊ GC (min … max): 0.00% … 97.55%
 Time  (median):     76.198 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   84.809 μs ± 175.311 μs  ┊ GC (mean ± σ):  7.37% ±  3.51%

  ▆█▇▇▆▅▅▄▃▂▂▁▁                                                ▂
  ███████████████▇▆▆▆▆▄▄▅▄▅▄▄▆▅▄▅▅▅▅▄▄▅▅▅▄▄▅▄▅▄▅▄▄▅▂▄▃▅▄▂▅▅▅▃▃ █
  73.2 μs       Histogram: log(frequency) by time       138 μs <

 Memory estimate: 64.45 KiB, allocs estimate: 1006.

### Optimization

In [42]:
@inline function larCellProd_opt(cellLists::Vector{Lar.Cells})::Lar.Cells
    shapes = [length(item) for item in cellLists]
    subscripts = cart_opt([collect(range(0, length=shape)) for shape in shapes])
    indices = [collect(tuple) .+ 1 for tuple in subscripts]
 
    jointCells = [cart_opt([cells[k] for (k,cells) in zip(index,cellLists)]) for index in indices]
    convertIt = index2addr_opt([ (length(cellLists[k][1]) > 1) ? shape .+ 1 : shape for (k,shape) in enumerate(shapes) ])
    [vcat(map(convertIt, map(collect,jointCells[j]))...) for j in 1:length(jointCells)]
end

larCellProd_opt (generic function with 1 method)

In [43]:
c1 = [[0,1],[1,2],[2,3]]
c0 = [[0],[1],[2]]
@benchmark larCellProd_opt([c1, c1, c0])

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  45.518 μs …   4.875 ms  ┊ GC (min … max):  0.00% … 98.00%
 Time  (median):     48.492 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   56.581 μs ± 169.601 μs  ┊ GC (mean ± σ):  11.50% ±  3.79%

   ▆██▄▁                                                        
  ▃█████▆▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  45.5 μs         Histogram: frequency by time         94.3 μs <

 Memory estimate: 67.36 KiB, allocs estimate: 651.

## filterByOrder( n::Int )Array{Array{Array{Int8,1},1},1}
Filtra l'array di codici binari di n bit in base alla somma dei loro valori interi.

La variabile `terms` è una collezione di tutte le possibili combinazioni (array) di valori `0,1` con n (valore in input) a disposizione.

Questa collezione è poi restituita in forma ordinata in base alla somma dei valori degli array contenenti le combinazioni.

Prima riga: somma = 1,
Seconda riga: somma =2,
    ...
N-esima riga = somma = n

In [44]:
function filterByOrder(n::Int)Array{Array{Array{Int8,1},1},1}
    terms = [[parse(Int8,bit) for bit in collect(term)] for term in Lar.binaryRange(n)]
    return [[term for term in terms if sum(term) == k] for k in 0:n]
end

filterByOrder (generic function with 1 method)

In [45]:
@benchmark filterByOrder(2)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.134 μs … 663.770 μs  ┊ GC (min … max):  0.00% … 99.61%
 Time  (median):     1.200 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   1.438 μs ±  11.176 μs  ┊ GC (mean ± σ):  13.42% ±  1.73%

  ▄▇█▇▆▆▄▃▃▂▂▁                                                ▂
  ████████████████▇▇▆▇▇▇▆▆▆▅▅▅▃▄▄▄▃▂▂▄▂▂▄▄▃▂▄▃▄▃▄▃▃▃▄▂▄▄▄▃▂▄▄ █
  1.13 μs      Histogram: log(frequency) by time      2.37 μs <

 Memory estimate: 2.00 KiB, allocs estimate: 40.

### Optimization

In [46]:
function filterByOrder_opt(n::Int)Vector{Vector{Vector{Int8}}}
    terms = [[parse(Int8,bit) for bit in collect(term)] for term in Lar.binaryRange(n)]
    return [[term for term in terms if sum(term) == k] for k in 0:n]
end

filterByOrder_opt (generic function with 1 method)

In [47]:
@benchmark filterByOrder_opt(2)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.126 μs … 631.822 μs  ┊ GC (min … max):  0.00% … 99.48%
 Time  (median):     1.191 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   1.426 μs ±  10.890 μs  ┊ GC (mean ± σ):  13.19% ±  1.72%

  ▅██▇▇▅▄▄▃▃▂▂▁▁                                              ▂
  ██████████████████▇▆▇▇▆▇▇▆▄▆▅▅▆▄▄▄▃▅▄▄▅▅▄▃▄▄▄▄▄▄▅▅▅▅▅▅▄▄▁▄▅ █
  1.13 μs      Histogram: log(frequency) by time      2.39 μs <

 Memory estimate: 2.00 KiB, allocs estimate: 40.

## larGridSkeleton( shape::Array{Int,1} )( d::Int )::Cells
Genera il `d`-scheletro di una griglia cuboidale con una data `shape`.

Ricordiamo che  il `d`-scheletro di un `p`-complesso cellulare (d<=p) è l'insieme delle `d`-celle che compongono il `p`-complesso. È quindi un suo sottocomplesso.

In [48]:
function larGridSkeleton(shape)
    n = length(shape)
    function larGridSkeleton0( d::Int )::Lar.Cells

    	@assert d<=n

        components = filterByOrder(n)[d .+ 1]
        apply(fun,a) = fun(a)
		componentCellLists = [ [
            map(f,x)  for (f,x) in  zip( [larGrid(dim) for dim in shape], convert(Array{Int64,1},component) )
        ] for component in components ]
        colList(arr) = [arr[:,k]  for k in 1:size(arr,2)]
        out = [ larCellProd(map(colList,cellLists)) for cellLists in componentCellLists ]
        return vcat(out...)
    end
    return larGridSkeleton0
end

larGridSkeleton (generic function with 1 method)

In [49]:
@benchmark larGridSkeleton([1,1,1])(3)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  11.125 μs …  6.168 ms  ┊ GC (min … max): 0.00% … 98.90%
 Time  (median):     12.078 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.296 μs ± 61.681 μs  ┊ GC (mean ± σ):  4.59% ±  0.99%

  ▃▇██▇▆▅▄▃▂▂▁                                                ▂
  ████████████████▆▇▇▇▇▆▇▆▄▅▅▅▄▅▅▄▄▄▅▄▅▄▁▄▃▄▄▄▅▄▃▃▅▅▄▅▄▄▄▅▄▄▄ █
  11.1 μs      Histogram: log(frequency) by time      27.1 μs <

 Memory estimate: 10.81 KiB, allocs estimate: 190.

### Optimization

In [50]:
@inline function larGridSkeleton_opt(shape::Vector{Int})
    n = length(shape)
    function larGridSkeleton0_opt( d::Int )::Lar.Cells

    	@assert d<=n

        components = filterByOrder_opt(n)[d .+ 1]
        apply(fun,a) = fun(a)
		componentCellLists = [ [
            map(f,x)  for (f,x) in  zip( [larGrid_opt(dim) for dim in shape], convert(Vector{Int64},component) )
        ] for component in components ]
        colList(arr) = [arr[:,k]  for k in 1:size(arr,2)]
        out = [ larCellProd_opt(map(colList,cellLists)) for cellLists in componentCellLists ]
        return vcat(out...)
    end
    return larGridSkeleton0_opt
end

larGridSkeleton_opt (generic function with 1 method)

In [51]:
@benchmark larGridSkeleton_opt([1,1,1])(3)

LoadError: MethodError: no method matching sort(::Tuple{Int64, Int64, Int64})
[0mClosest candidates are:
[0m  sort([91m::AbstractUnitRange[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/range.jl:1304
[0m  sort([91m::AbstractRange[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/range.jl:1307
[0m  sort([91m::SparseArrays.SparseVector{Tv, Ti}[39m; kws...) where {Tv, Ti} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/SparseArrays/src/sparsevector.jl:1968
[0m  ...

## larImageVerts(shape::Array{Int,1})::Array{Int64,2}
Linearizza la *grid of integer vertices*, data la `shape` di una *griglia cuboidale*.

In [52]:
function larImageVerts( shape::Array{Int,1} )::Array{Int64,2}
    vertexDomain(n) = hcat([k for k in 0:n-1]...)
    vertLists = [vertexDomain(k+1) for k in shape]
    vertGrid = larVertProd(vertLists)
    return vertGrid
end

larImageVerts (generic function with 1 method)

In [53]:
@benchmark larImageVerts([2,2,2])

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   8.390 μs …  1.570 ms  ┊ GC (min … max):  0.00% … 98.02%
 Time  (median):      8.969 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   10.478 μs ± 41.264 μs  ┊ GC (mean ± σ):  11.00% ±  2.78%

  ▃▇██▇▆▅▄▃▃▂▁▁ ▁▁▁                                           ▂
  ██████████████████▇▇▇▅▅▅▅▅▄▃▄▅▄▅▄▄▅▁▅▄▃▃▄▄▄▅▁▄▃▃▄▄▃▄▄▄▃▄▅▄▄ █
  8.39 μs      Histogram: log(frequency) by time      18.5 μs <

 Memory estimate: 12.11 KiB, allocs estimate: 120.

### Optimization

In [54]:
function larImageVerts_opt( shape::Vector{Int} )::Matrix{Int64}
    vertexDomain(n) = hcat([k for k in 0:n-1]...)
    vertLists = [vertexDomain(k+1) for k in shape]
    vertGrid = larVertProd_opt(vertLists)
    return vertGrid
end

larImageVerts_opt (generic function with 1 method)

In [55]:
@benchmark larImageVerts_opt([2,2,2])

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   8.313 μs …  1.554 ms  ┊ GC (min … max):  0.00% … 98.74%
 Time  (median):      8.894 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   10.460 μs ± 41.059 μs  ┊ GC (mean ± σ):  10.97% ±  2.78%

  ▄▇█▇▆▅▄▃▂▂▁▁▁▁                                              ▂
  ████████████████▇▆▆▆▄▅▆▄▅▄▄▃▄▅▄▄▅▁▅▄▅▅▅▃▄▄▆▅▅▄▅▅▅▅▅▃▄▃▃▅▅▄▅ █
  8.31 μs      Histogram: log(frequency) by time      20.4 μs <

 Memory estimate: 12.11 KiB, allocs estimate: 120.

## cuboidGrid( shape, filled=false )::Union( Cells, Array{Cells,1} )
Funzione generatrice multidimensionale. 
Genera o una griglia ``d``-dimensionale solida di ``d``-cuboidi nello spazio ``d``-dimensionale oppure l’array dello scheletro di dimensione ``p``, questo dipende dalla variabile Booleana `filled` in input. ``0``-cuboidi sono punti, ``1``-cuboidi sono segmenti, ``2``-cuboidi sono quadrati,  ``3``-cuboidi sono cubi, etc. I valori di `shape=[a,b,c]` determinano il numero ``a x b x c`` di ``d``-cellulari. Inoltre, `d = length(shape)`.

In [56]:
function cuboidGrid( shape, filled=false )
   vertGrid = larImageVerts(shape)
   gridMap = larGridSkeleton(shape)
   if ! filled
      cells = gridMap(length(shape))
   else
      skeletonIds = 0:length(shape)
      cells = [ gridMap(id) for id in skeletonIds ]
   end
   return convert(Array{Float64,2},vertGrid), cells
end

cuboidGrid (generic function with 2 methods)

In [57]:
@benchmark cuboidGrid([3,2,1],true)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  322.298 μs …   7.772 ms  ┊ GC (min … max): 0.00% … 92.93%
 Time  (median):     336.635 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   385.182 μs ± 475.020 μs  ┊ GC (mean ± σ):  9.14% ±  7.02%

  ▇██▇▆▅▄▃▃▂▂▂▂▂▂▁▂▂▂▂▂▁▁ ▁                                     ▂
  ██████████████████████████▇█▇▇▇▇▇▆▅▅▆▅▅▅▃▆▅▃▃▆▁▄▄▁▅▄▁▄▅▁▃▁▁▃▅ █
  322 μs        Histogram: log(frequency) by time        639 μs <

 Memory estimate: 258.34 KiB, allocs estimate: 4206.

### Optimization

In [58]:
function cuboidGrid_opt( shape, filled=false )
   vertGrid = larImageVerts_opt(shape)
   gridMap = larGridSkeleton_opt(shape)
   if ! filled
      cells = gridMap(length(shape))
   else
      skeletonIds = 0:length(shape)
      cells = [ gridMap(id) for id in skeletonIds ]
   end
   return convert(Matrix{Float64},vertGrid), cells
end

cuboidGrid_opt (generic function with 2 methods)

In [96]:
@benchmark cuboidGrid_opt([3,2,1],true)

LoadError: MethodError: no method matching sort(::Tuple{Int64, Int64, Int64})
[0mClosest candidates are:
[0m  sort([91m::AbstractUnitRange[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/range.jl:1304
[0m  sort([91m::AbstractRange[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/range.jl:1307
[0m  sort([91m::SparseArrays.SparseVector{Tv, Ti}[39m; kws...) where {Tv, Ti} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/SparseArrays/src/sparsevector.jl:1968
[0m  ...

## larModelProduct(modelOne, modelTwo)::Lar.LAR
La funzione prende in input due *Modelli LAR* e ne restituisce il loro *prodotto Cartesiano*.
Un *Modello Lar* è una coppia Geometria, Topologia dove la Geometria descrive come l'oggetto è immerso nello spazio mentre la Topologia è l'insieme di quelle relazioni come spigoli forniti attraverso i vertici, facce fornite attraverso i vertici e celle tridimensionali fornite attraverso i vertici.
Fondamentalmente un *Modello Lar* è una coppia di oggetti di coordinate e relazioni di adiacenza e incidenza tra celle.

In [60]:
function larModelProduct( modelOne, modelTwo )
    (V, cells1) = modelOne
    (W, cells2) = modelTwo

    vertices = DataStructures.OrderedDict();
    k = 1
    for j in 1:size(V,2)
        v = V[:,j]
        for i in 1:size(W,2)
            w = W[:,i]
            id = [v;w]
            if haskey(vertices, id) == false
                vertices[id] = k
                k = k + 1
            end
        end
    end

    cells = []
    for c1 in cells1
        for c2 in cells2
            cell = []
            for vc in c1
                for wc in c2
                    push!(cell, vertices[[V[:,vc];W[:,wc]]] )
                end
            end
            push!(cells, cell)
        end
    end

    vertexmodel = []
    for v in keys(vertices)
        push!(vertexmodel, v)
    end
    verts = hcat(vertexmodel...)
    cells = [[v for v in cell] for cell in cells]
    return (verts, cells)
end

function larModelProduct( twoModels )
    modelOne, modelTwo = twoModels
    larModelProduct(modelOne, modelTwo)
end

larModelProduct (generic function with 2 methods)

In [61]:
geom,topol = [0. 1. 2.], [[1,2],[2,3]]
mod = (geom,topol)
@benchmark larModelProduct(mod, mod)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  10.885 μs …  7.017 ms  ┊ GC (min … max):  0.00% … 99.24%
 Time  (median):     11.573 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   13.340 μs ± 98.607 μs  ┊ GC (mean ± σ):  10.37% ±  1.40%

   ▄█▇▄                                                        
  ▄█████▆▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  10.9 μs         Histogram: frequency by time        22.3 μs <

 Memory estimate: 9.00 KiB, allocs estimate: 139.

### Optimization

In [62]:
@inline function createCells(V,cells1,W,cells2,vertices)
    cells = []
    for c1 in cells1
        for c2 in cells2
            cell = []
            @inbounds for vc in c1
                @inbounds @simd for wc in c2
                    push!(cell, vertices[[V[:,vc];W[:,wc]]] )
                end
            end
            push!(cells, cell)
        end
    end
    return cells
end

@inline function createVertices(V,W)
    vertices = DataStructures.OrderedDict();
    k = 1
    @inbounds for j in 1:size(V,2)
        v = V[:,j]
        @inbounds @simd for i in 1:size(W,2)
            w = W[:,i]
            id = [v;w]
            if haskey(vertices, id) == false
                vertices[id] = k
                k = k + 1
            end
        end
    end
    return vertices
end


function larModelProduct_opt( modelOne, modelTwo )
    (V, cells1) = modelOne
    (W, cells2) = modelTwo

    vertices = createVertices(V,W)

    verts = reduce(hcat, keys(vertices))
    cells = [[v for v in cell] for cell in createCells(V,cells1,W,cells2,vertices)]
    return (verts, cells)
end

function larModelProduct_opt( twoModels )
    modelOne, modelTwo = twoModels
    larModelProduct_opt(modelOne, modelTwo)
end

larModelProduct_opt (generic function with 2 methods)

In [63]:
geom,topol = [0. 1. 2.], [[1,2],[2,3]]
mod = (geom,topol)
@benchmark larModelProduct_opt(mod, mod)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  10.829 μs …  6.916 ms  ┊ GC (min … max): 0.00% … 99.34%
 Time  (median):     12.133 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.942 μs ± 69.512 μs  ┊ GC (mean ± σ):  4.31% ±  0.99%

  ▆█▇▅▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁                                     ▁
  ████████████████████████████▇▆▆▇▇▇▇▇██▇▆▆▅▆▆▅▅▆▆▆▆▆▅▅▄▄▅▅▄▄ █
  10.8 μs      Histogram: log(frequency) by time      48.4 μs <

 Memory estimate: 9.22 KiB, allocs estimate: 139.

## INSR(f::Function)(seq::Array{Any,1})::Any
Trasforma una funzione binaria in una funzione n-aria.

In [64]:
function INSR(f)
	function INSR0(seq)
		len = length(seq)
		res = seq[end]
		for i in range(len-2,step=-1,stop=0)
			res = f([seq[i+1], res])
		end
		return res
	end
	return INSR0
end

INSR (generic function with 1 method)

In [65]:
mod1D = grid(repeat([.1,-.1],outer=5)...)
@benchmark INSR(larModelProduct)([mod1D,mod1D])

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   96.903 μs …   7.420 ms  ┊ GC (min … max): 0.00% … 98.07%
 Time  (median):      99.594 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   113.188 μs ± 270.845 μs  ┊ GC (mean ± σ):  9.20% ±  3.79%

  ▆█▇▆▅▄▄▃▃▂▂▁                                                  ▂
  █████████████▇▇▇▇▆▅▆▆▅▅▆▆▆▆▆▅▆▅▅▅▅▆▅▆▅▅▅▆▅▅▆▅▅▅▅▅▅▅▅▅▅▅▄▅▄▅▄▄ █
  96.9 μs       Histogram: log(frequency) by time        165 μs <

 Memory estimate: 69.00 KiB, allocs estimate: 1073.

### Optimization

In [66]:
function INSR_opt(f)
	function INSR0_opt(seq)
		len = length(seq)
		res = seq[end]
		@inbounds for i in range(len-2,step=-1,stop=0)
			res = f([seq[i+1], res])
		end
		return res
	end
	return INSR0_opt
end

INSR_opt (generic function with 1 method)

In [67]:
mod1D = grid(repeat([.1,-.1],outer=5)...)
@benchmark INSR_opt(larModelProduct)([mod1D,mod1D])

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   96.836 μs …   7.503 ms  ┊ GC (min … max): 0.00% … 98.21%
 Time  (median):      99.255 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   112.573 μs ± 271.076 μs  ┊ GC (mean ± σ):  9.26% ±  3.79%

  ▆█▇▆▅▄▄▃▂▂▁                                                   ▁
  █████████████▇▇▇▇▆▆▆▆▅▆▆▆▆▆▆▆▅▆▅▃▆▆▄▅▅▅▅▅▃▅▄▅▄▅▄▄▅▆▄▅▄▅▄▄▅▅▅▄ █
  96.8 μs       Histogram: log(frequency) by time        164 μs <

 Memory estimate: 69.00 KiB, allocs estimate: 1073.